In [1]:
import numpy as np
import pandas as pd
import os
import calendar
import datetime

In [2]:
# Importing static dataset from .csv file
raw_data = pd.read_csv('datasets/Motor_Vehicle_Collisions_-_Crashes.csv')

print(raw_data.shape)
raw_data.head()

(904482, 29)


CRASH DATE CRASH TIME    BOROUGH  ZIP CODE  LATITUDE  LONGITUDE  \
0  04/02/2016      22:23      BRONX   10456.0       NaN        NaN   
1  01/06/2020      12:50        NaN       NaN       NaN        NaN   
2  01/06/2020      13:16  MANHATTAN   10018.0       NaN        NaN   
3  01/06/2020      16:00        NaN       NaN  40.73937  -73.79189   
4  04/02/2016       7:30  MANHATTAN   10007.0       NaN        NaN   

                     LOCATION                    ON STREET NAME  \
0                         NaN  WESTCHESTER AVENUE                 
1                         NaN  PRINCE STREET                      
2                         NaN  40 STREET                          
3  POINT (-73.79189 40.73937)  LONG ISLAND EXPRESSWAY             
4                         NaN  DEY STREET                         

  CROSS STREET NAME OFF STREET NAME  ...  CONTRIBUTING FACTOR VEHICLE 2  \
0      UNION AVENUE             NaN  ...                            NaN   
1               NaN             NaN  ...  Failure to Yield Right-of-Way   
2         12 AVENUE             NaN  ...                    Unspecified   
3               NaN             NaN  ...                    Unspecified   
4     CHURCH STREET             NaN  ...                    Unspecified   

   CONTRIBUTING FACTOR VEHICLE 3  CONTRIBUTING FACTOR VEHICLE 4  \
0                            NaN                            NaN   
1                            NaN                            NaN   
2                            NaN                            NaN   
3                            NaN                            NaN   
4                            NaN                            NaN   

   CONTRIBUTING FACTOR VEHICLE 5  COLLISION_ID  VEHICLE TYPE CODE 1  \
0                            NaN     3411047.0    PASSENGER VEHICLE   
1                            NaN     4269674.0                Sedan   
2                            NaN     4269909.0                Sedan   
3                            NaN     4269644.0                Sedan   
4                            NaN     3405353.0    PASSENGER VEHICLE   

    VEHICLE TYPE CODE 2  VEHICLE TYPE CODE 3 VEHICLE TYPE CODE 4  \
0                   NaN                  NaN                 NaN   
1  Tractor Truck Diesel                  NaN                 NaN   
2                   Bus                  NaN                 NaN   
3                 Sedan                  NaN                 NaN   
4                   NaN                  NaN                 NaN   

  VEHICLE TYPE CODE 5  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  

[5 rows x 29 columns]

In [3]:
# Removing columns that have more than a third values as NaN
mask = raw_data.isna().sum() / len(raw_data) < 0.34
raw_data = raw_data.loc[:, mask]

# Removing columns that don't have a large contributing factor to EDA and Predictions
cols_to_drop = ['ZIP CODE', 'LOCATION', 
                'CONTRIBUTING FACTOR VEHICLE 2', 'VEHICLE TYPE CODE 2']
raw_data.drop(cols_to_drop, axis = 1, inplace = True)

In [4]:
# Concatenating date and time columns
raw_data['CRASH_DATE_TIME'] = raw_data['CRASH DATE'] + ' ' + raw_data['CRASH TIME']

# Drop redundant date and time columns
cols_to_drop = ['CRASH DATE', 'CRASH TIME']
raw_data.drop(cols_to_drop, axis = 1, inplace = True)

In [5]:
# Convert Crash Date and time to datetime format
raw_data['CRASH_DATE_TIME']= pd.to_datetime(raw_data['CRASH_DATE_TIME'], 
                                            dayfirst=True, errors='coerce')

# Dropping rows with problematics dates
idx = raw_data[raw_data['CRASH_DATE_TIME'].isnull()].index
raw_data.drop(idx, axis = 0, inplace = True)

print(raw_data.shape)
raw_data.head(3)

(904477, 16)


BOROUGH  LATITUDE  LONGITUDE                    ON STREET NAME  \
0      BRONX       NaN        NaN  WESTCHESTER AVENUE                 
1        NaN       NaN        NaN  PRINCE STREET                      
2  MANHATTAN       NaN        NaN  40 STREET                          

   NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
0                        1.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   

   NUMBER OF PEDESTRIANS INJURED  NUMBER OF PEDESTRIANS KILLED  \
0                            0.0                           0.0   
1                            0.0                           0.0   
2                            0.0                           0.0   

   NUMBER OF CYCLIST INJURED  NUMBER OF CYCLIST KILLED  \
0                        0.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   

   NUMBER OF MOTORIST INJURED  NUMBER OF MOTORIST KILLED  \
0                         1.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   

      CONTRIBUTING FACTOR VEHICLE 1  COLLISION_ID VEHICLE TYPE CODE 1  \
0    Driver Inattention/Distraction     3411047.0   PASSENGER VEHICLE   
1  Lane Marking Improper/Inadequate     4269674.0               Sedan   
2             Following Too Closely     4269909.0               Sedan   

      CRASH_DATE_TIME  
0 2016-02-04 22:23:00  
1 2020-06-01 12:50:00  
2 2020-06-01 13:16:00

In [6]:
# Dropping columns
cols_to_drop_again = ['LATITUDE', 'LONGITUDE', 'ON STREET NAME', 'COLLISION_ID']
raw_data.drop(cols_to_drop_again, axis = 1, inplace = True)

print(raw_data.shape)
raw_data.head()

(904477, 12)


BOROUGH  NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
0      BRONX                        1.0                       0.0   
1        NaN                        0.0                       0.0   
2  MANHATTAN                        0.0                       0.0   
3        NaN                        0.0                       0.0   
4  MANHATTAN                        0.0                       0.0   

   NUMBER OF PEDESTRIANS INJURED  NUMBER OF PEDESTRIANS KILLED  \
0                            0.0                           0.0   
1                            0.0                           0.0   
2                            0.0                           0.0   
3                            0.0                           0.0   
4                            0.0                           0.0   

   NUMBER OF CYCLIST INJURED  NUMBER OF CYCLIST KILLED  \
0                        0.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   
3                        0.0                       0.0   
4                        0.0                       0.0   

   NUMBER OF MOTORIST INJURED  NUMBER OF MOTORIST KILLED  \
0                         1.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   
3                         0.0                        0.0   
4                         0.0                        0.0   

      CONTRIBUTING FACTOR VEHICLE 1 VEHICLE TYPE CODE 1     CRASH_DATE_TIME  
0    Driver Inattention/Distraction   PASSENGER VEHICLE 2016-02-04 22:23:00  
1  Lane Marking Improper/Inadequate               Sedan 2020-06-01 12:50:00  
2             Following Too Closely               Sedan 2020-06-01 13:16:00  
3    Driver Inattention/Distraction               Sedan 2020-06-01 16:00:00  
4                       Unspecified   PASSENGER VEHICLE 2016-02-04 07:30:00

In [7]:
# Removing time from date time column.
raw_data['CRASH_DATE'] = pd.to_datetime(raw_data['CRASH_DATE_TIME']).dt.date
raw_data.head()

BOROUGH  NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
0      BRONX                        1.0                       0.0   
1        NaN                        0.0                       0.0   
2  MANHATTAN                        0.0                       0.0   
3        NaN                        0.0                       0.0   
4  MANHATTAN                        0.0                       0.0   

   NUMBER OF PEDESTRIANS INJURED  NUMBER OF PEDESTRIANS KILLED  \
0                            0.0                           0.0   
1                            0.0                           0.0   
2                            0.0                           0.0   
3                            0.0                           0.0   
4                            0.0                           0.0   

   NUMBER OF CYCLIST INJURED  NUMBER OF CYCLIST KILLED  \
0                        0.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   
3                        0.0                       0.0   
4                        0.0                       0.0   

   NUMBER OF MOTORIST INJURED  NUMBER OF MOTORIST KILLED  \
0                         1.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   
3                         0.0                        0.0   
4                         0.0                        0.0   

      CONTRIBUTING FACTOR VEHICLE 1 VEHICLE TYPE CODE 1     CRASH_DATE_TIME  \
0    Driver Inattention/Distraction   PASSENGER VEHICLE 2016-02-04 22:23:00   
1  Lane Marking Improper/Inadequate               Sedan 2020-06-01 12:50:00   
2             Following Too Closely               Sedan 2020-06-01 13:16:00   
3    Driver Inattention/Distraction               Sedan 2020-06-01 16:00:00   
4                       Unspecified   PASSENGER VEHICLE 2016-02-04 07:30:00   

   CRASH_DATE  
0  2016-02-04  
1  2020-06-01  
2  2020-06-01  
3  2020-06-01  
4  2016-02-04

In [8]:
# Removing that column.
cols_to_drop_again = ['CRASH_DATE_TIME']
raw_data.drop(cols_to_drop_again, axis = 1, inplace = True)

raw_data.head()

BOROUGH  NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
0      BRONX                        1.0                       0.0   
1        NaN                        0.0                       0.0   
2  MANHATTAN                        0.0                       0.0   
3        NaN                        0.0                       0.0   
4  MANHATTAN                        0.0                       0.0   

   NUMBER OF PEDESTRIANS INJURED  NUMBER OF PEDESTRIANS KILLED  \
0                            0.0                           0.0   
1                            0.0                           0.0   
2                            0.0                           0.0   
3                            0.0                           0.0   
4                            0.0                           0.0   

   NUMBER OF CYCLIST INJURED  NUMBER OF CYCLIST KILLED  \
0                        0.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   
3                        0.0                       0.0   
4                        0.0                       0.0   

   NUMBER OF MOTORIST INJURED  NUMBER OF MOTORIST KILLED  \
0                         1.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   
3                         0.0                        0.0   
4                         0.0                        0.0   

      CONTRIBUTING FACTOR VEHICLE 1 VEHICLE TYPE CODE 1  CRASH_DATE  
0    Driver Inattention/Distraction   PASSENGER VEHICLE  2016-02-04  
1  Lane Marking Improper/Inadequate               Sedan  2020-06-01  
2             Following Too Closely               Sedan  2020-06-01  
3    Driver Inattention/Distraction               Sedan  2020-06-01  
4                       Unspecified   PASSENGER VEHICLE  2016-02-04

In [9]:
# Removing null values from BOROUGH column.
raw_data = raw_data.dropna(subset = ['BOROUGH', 'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED'])
print(raw_data.shape)
raw_data.head()

(595125, 12)


BOROUGH  NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
0      BRONX                        1.0                       0.0   
2  MANHATTAN                        0.0                       0.0   
4  MANHATTAN                        0.0                       0.0   
6      BRONX                        0.0                       0.0   
8   BROOKLYN                        0.0                       0.0   

   NUMBER OF PEDESTRIANS INJURED  NUMBER OF PEDESTRIANS KILLED  \
0                            0.0                           0.0   
2                            0.0                           0.0   
4                            0.0                           0.0   
6                            0.0                           0.0   
8                            0.0                           0.0   

   NUMBER OF CYCLIST INJURED  NUMBER OF CYCLIST KILLED  \
0                        0.0                       0.0   
2                        0.0                       0.0   
4                        0.0                       0.0   
6                        0.0                       0.0   
8                        0.0                       0.0   

   NUMBER OF MOTORIST INJURED  NUMBER OF MOTORIST KILLED  \
0                         1.0                        0.0   
2                         0.0                        0.0   
4                         0.0                        0.0   
6                         0.0                        0.0   
8                         0.0                        0.0   

    CONTRIBUTING FACTOR VEHICLE 1 VEHICLE TYPE CODE 1  CRASH_DATE  
0  Driver Inattention/Distraction   PASSENGER VEHICLE  2016-02-04  
2           Following Too Closely               Sedan  2020-06-01  
4                     Unspecified   PASSENGER VEHICLE  2016-02-04  
6                     Unspecified   PASSENGER VEHICLE  2016-01-04  
8           Following Too Closely   PASSENGER VEHICLE  2016-01-04

In [10]:
# Saving the dataset.
raw_data = raw_data.reset_index()
raw_data.to_csv('datasets/Motor_Vehicle_Collisions_-_Crashes_Clean.csv')

In [11]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595125 entries, 0 to 595124
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   index                          595125 non-null  int64  
 1   BOROUGH                        595125 non-null  object 
 2   NUMBER OF PERSONS INJURED      595125 non-null  float64
 3   NUMBER OF PERSONS KILLED       595125 non-null  float64
 4   NUMBER OF PEDESTRIANS INJURED  595125 non-null  float64
 5   NUMBER OF PEDESTRIANS KILLED   595125 non-null  float64
 6   NUMBER OF CYCLIST INJURED      595125 non-null  float64
 7   NUMBER OF CYCLIST KILLED       595125 non-null  float64
 8   NUMBER OF MOTORIST INJURED     595125 non-null  float64
 9   NUMBER OF MOTORIST KILLED      595125 non-null  float64
 10  CONTRIBUTING FACTOR VEHICLE 1  595125 non-null  object 
 11  VEHICLE TYPE CODE 1            595125 non-null  object 
 12  CRASH_DATE                    